In [ ]:
# # Linear Regression for top_hundred
# a, b = np.polyfit(top_hundred.Age, top_hundred.Place, 1)
# plt.scatter(x=top_hundred.Age, y=top_hundred.Place)
# plt.plot(top_hundred.Age, a*top_hundred.Age+b, color = 'darkorange')
# plt.show()

In [ ]:
# # Linear Regression for adult_df
# a, b = np.polyfit(adult_df.Age, adult_df.Place, 1)
# plt.scatter(x=adult_df.Age, y=adult_df.Place)
# plt.plot(adult_df.Age, a*adult_df.Age+b, color = 'darkorange')
# plt.show()

In [86]:
res = sts.linregress(adult_df.Age, adult_df.Place)
print(f"R-squared: {res.rvalue**2:.6f}")

R-squared: 0.041883


# Smooth Modelling

In [80]:
# Remove missing values
adult_df = adult_df.dropna()

# Reset index
adult_df = adult_df.reset_index(drop=True)

# Assign variable to x and y
x = adult_df.Age
y = adult_df.Place

# Remove duplicates
unique_x, unique_idx = np.unique(x, return_index=True)
x = unique_x
y = y[unique_idx]

# Define custom index
idx = ['point_{}'.format(i) for i in range(len(x))]

# Convert the lists to pandas Series with custom index 'idx'
x = pd.Series(x, index=idx)
y = pd.Series(y, index=idx)

# Sort the Series by index
x_sorted = x.sort_index()
y_sorted = y.sort_index()

print(np.isnan(y_sorted).any())  # Check for NaN values
print(np.isinf(y_sorted).any())  # Check for Inf values

True
False


In [84]:
# If either is 'True' remove the corresponding elements from the 'unique_x' and 'y_sorted' using boolean indexing
mask = ~(np.isnan(y_sorted) | np.isinf(y_sorted))
unique_x = unique_x[mask]
y_sorted = y_sorted[mask]

### It seems that the array unique_x is empty, which causes an IndexError when trying to access its first element. This is most likely because all values in the Age column of the adult_df dataframe were duplicates and were removed by the unique() function. You may want to check if this is the case and consider using a different dataset if you want to explore the relationship between age and place.

In [83]:
# Fit a curve to the data
spline = make_interp_spline(unique_x, y_sorted)
x_smooth = np.linspace(x_sorted.min(), x_sorted.max(), 100)
y_smooth = spline(x_smooth)

# Plot the scatter plot and the smooth curve
plt.scatter(x, y)
plt.plot(x_smooth, y_smooth)

# Show the plot
plt.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [68]:
top_hundred.Age.describe()

count    100.000000
mean      26.090000
std        4.062877
min       18.000000
25%       23.000000
50%       26.000000
75%       28.000000
max       40.000000
Name: Age, dtype: float64

In [69]:
top_ten = adult_df[adult_df.Place <= 10]
top_ten.Age.describe()

count    10.00000
mean     28.10000
std       3.28126
min      25.00000
25%      26.00000
50%      27.00000
75%      29.50000
max      35.00000
Name: Age, dtype: float64

## Parse html for seasons bests

In [18]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# Define the base URL and parameters for the rankings page
base_url = 'https://worldathletics.org'
params = {
    'regionType': 'world',
    'page': 1,
    'rankDate': '2023-05-09',
    'limitByCountry': 0,
    'eventId': '4f558bc9-9b95-4b5a-b80a-4d0f381bffb2',  # Javelin throw event ID
}

# Send a GET request to the rankings page and parse the response with BeautifulSoup
response = requests.get(base_url + '/world-rankings/javelin-throw/men', params=params)
soup = soup(response.content, 'html.parser')
soup

[]

In [ ]:
### Note: website was not scraped successfully. print(soup) yielded empty list.

In [17]:
# Find the table of athletes and loop through each row to extract the athlete's profile link
athlete_links = []
table = soup.find('table', {'class': 'table-athletes'})
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if cells:
        link = cells[1].find('a')['href']
        athlete_links.append(link)

# Loop through each athlete's profile page and extract their season's best throw
season_bests = []
for link in athlete_links:
    response = requests.get(base_url + link)
    soup = BeautifulSoup(response.content, 'html.parser')
    results_table = soup.find('table', {'class': 'table-res'})
    season_bests.append(results_table.find_all('td', {'class': 'seasonBest'})[0].text)

# Create a pandas DataFrame with the athlete's name and season's best throw
df = pd.DataFrame({'Name': [link.split('/')[-1] for link in athlete_links], 'Season Best': season_bests})

AttributeError: 'NoneType' object has no attribute 'find_all'